In [1]:
using LinearAlgebra, SparseArrays
using Arpack
import gmsh
gmsh.initialize()

include("../src/FEM.jl")
using .FEM

In [2]:
# Adatok
E = 2e5
ν = 0.3
ρ = 7.85e-9
thickness = 1
height = 10
base = 100
elemSize = 2 #22

approxOrder = 2
internalNodes = true
quadElements = true

true

In [3]:
# Geometria megrajzolása
gmsh.model.add("rectangle")

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
#p23 = gmsh.model.occ.addPoint(base, height / 2, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
#l23 = gmsh.model.occ.addLine(p23, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

l5 = gmsh.model.occ.addCircle(base / 2, height / 2, 0, min(base, height) / 4)
cl2 = gmsh.model.occ.addCurveLoop([l5])

sf1 = gmsh.model.occ.addPlaneSurface([cl1, cl2])
#display(sf1)

#cp1 = gmsh.model.occ.addPoint(base / 2, height / 2, 0)
#cl1 = gmsh.model.occ.addLine(p1, p3)
#cl2 = gmsh.model.occ.addLine(p2, p4)

#sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, cl1)])
#gmsh.model.occ.removeAllDuplicates()

#p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
#p6 = gmsh.model.occ.addPoint(base, height / 2, 0)
#l5 = gmsh.model.occ.addLine(p5, p6)

gmsh.model.occ.synchronize()
#sf2

In [4]:
problem = FEM.Problem(type="PlaneStress", E=E, ν=ν, ρ=ρ, thickness=thickness)

Main.FEM.Problem("rectangle", "PlaneStress", 2, 200000.0, 0.3, 7.85e-9, 1.0, 0)

In [5]:
phg = gmsh.model.addPhysicalGroup(1, [l2])
gmsh.model.setPhysicalName(1, phg, "supp")
phg = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, phg, "load")
phg = gmsh.model.addPhysicalGroup(2, [sf1])
gmsh.model.setPhysicalName(2, phg, "body")


In [6]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
load = FEM.load("load", fx=1, fy=0)

("load", 1, 0, 0)

In [7]:
FEM.generateMesh(problem, sf1, elemSize, approxOrder=approxOrder, algorithm=6, quadrangle=quadElements, internalNodes=internalNodes)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Circle)
Info    : Done meshing 1D (Wall 0.000431058s, CPU 0.000409s)
Info    : 118 nodes 123 elements
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.022223s, CPU 0.022189s)
Info    : 359 nodes 723 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 841 internal 118 closed
Info    : Blossom recombination completed (Wall 0.0127734s, CPU 0.012771s): 294 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.799105, min Q = 0.508151
Info    : Done recombining 2D mesh (Wall 0.0128315s, CPU 0.012832s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 20%] Meshing curve 2 order 2
Info    : [ 40%] Meshing curve 3 order 2
Info    : [ 50%] Meshing

In [8]:
ENV["LIBGL_ALWAYS_SOFTWARE"] = "true"
#gmsh.fltk.run()
#gmsh.fltk.wait(3)

"true"

In [9]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [10]:
K = FEM.stiffnessMatrix(problem)

2588×2588 SparseMatrixCSC{Float64, Int64} with 77152 stored entries:
⎡⣻⣾⣳⣍⠀⠁⢈⢿⣸⢸⡓⢰⣸⠨⡥⠶⠎⠦⠒⠂⠀⠐⢶⡰⣜⣓⣚⠀⠈⠏⢼⡠⡁⠀⠀⠀⠀⠌⢌⠄⎤
⎢⡝⢾⠱⣦⡀⣀⠀⠹⣼⢸⢉⠸⣼⡐⡖⠚⠃⠓⠉⢁⡀⠈⠳⠜⠮⡩⢭⠀⠀⠃⡚⡔⡄⢀⠀⠀⠀⠂⢢⡆⎥
⎢⠄⠀⠀⢨⣻⢞⡳⡅⡷⡃⢬⡳⣿⠀⣟⢼⡣⣋⡁⠠⣝⠲⠀⣉⠀⡇⠠⢈⠀⠀⠀⣟⣽⠄⠀⠀⠀⢘⡳⠀⎥
⎢⣦⣔⣄⡀⠝⠮⠱⣦⢹⣥⣄⣹⣽⣠⣯⣾⣵⣁⣀⡄⠈⢙⣀⣀⣲⣧⣤⣦⠠⣶⣀⣏⢾⣆⣀⠀⢀⠼⠹⡂⎥
⎢⣒⣚⣒⣛⠽⠫⠗⣶⣿⣿⣊⡟⣿⣻⣿⣾⣷⣿⠁⡈⠹⠛⣀⠋⢛⡋⢙⣚⣿⣿⣿⣿⣸⠉⠐⠛⣿⣻⣿⣟⎥
⎢⢙⣈⣃⡐⢦⡳⣄⣹⣮⠼⣻⣾⢿⣿⣭⡉⠌⠠⣟⡛⠦⣴⣂⣶⣾⣿⡛⡿⠤⠤⢢⠲⣘⣶⣾⣻⢯⣏⢇⣅⎥
⎢⡒⡚⢒⠻⠛⠛⠓⣻⣿⣻⣿⣷⣿⣿⡛⣚⣒⣃⠀⡽⢃⡼⣺⢿⡿⣝⣿⣿⣘⣛⣟⣲⣾⣾⢿⣦⣿⣟⣗⣛⎥
⎢⢡⡏⣸⠉⣛⣝⣫⣿⣻⣿⡇⠻⣻⢨⣿⣿⣭⣹⠀⠐⢘⠁⣵⠀⠛⢚⠘⠛⢿⣻⠿⡟⠉⡀⠀⠀⠛⡟⡟⠛⎥
⎢⠪⡅⢭⠀⡭⢪⠕⢻⣽⣿⠂⡁⠼⢸⣇⣻⣿⢟⡀⠀⡨⠑⠈⠀⠀⠐⠀⢀⢇⢿⣪⢫⠴⠂⠀⠀⢉⡲⣝⣙⎥
⎢⠸⠀⠇⢀⠁⡈⠀⠼⡁⠠⣿⠹⣄⡤⢀⠀⠀⠈⣿⣿⡀⠰⠇⢨⠰⡠⣧⡯⠀⠀⡀⡠⠀⢀⢀⣠⡄⢄⡀⢀⎥
⎢⢀⠀⡀⠈⢳⡙⣆⢀⣷⠂⢈⣧⣉⡴⠖⠐⢆⠊⢀⡈⡻⣮⡀⢉⡉⣏⢉⣈⠀⠒⠁⠲⢈⢥⢀⣤⠶⠀⠒⠆⎥
⎢⢘⡳⣙⠆⡄⢠⠀⢸⡤⠘⢨⣼⣾⣞⠑⠛⠂⠀⡉⣁⡄⢈⣻⣾⣾⣤⡄⣨⡠⠣⡙⠙⠰⣲⣖⢀⡐⠀⠀⠃⎥
⎢⢶⢹⡎⡣⠤⠤⠼⣾⡿⠰⣾⣿⣟⢯⣻⢀⢀⠀⠐⡢⡧⢬⠚⣿⣻⣾⣷⣛⢉⣐⣉⠂⢊⠑⠣⠤⠈⢙⠚⡛⎥
⎢⠚⠘⠃⠓⡀⢂⠠⣿⣳⢰⣿⡬⣿⣿⣶⠀⠀⢀⡭⡿⡃⢰⡀⣩⣽⢻⢿⣷⡀⠀⢓⣀⠎⣱⣬⠭⢷⣶⢴⣶⎥
⎢⡦⠄⠤⠀⠀⠀⢠⣦⣿⣿⠀⡇⣶⢸⣿⣳⣭⣕⠀⠀⢠⠀⠤⡊⢃⢰⠀⠈⣿⣿⣼⡍⣈⠂⠀⠀⠙⢶⣷⣅⎥
⎢⠒⡳⢚⠬⣤⢤⡤⢼⣿⣿⢨⡒⢻⣹⣿⠧⡮⣚⠀⡨⢡⡀⣗⠈⠣⠘⠙⢰⡖⠿⣛⣼⣾⣍⠉⠀⣶⡶⣼⡏⎥
⎢⠁⠈⠀⢉⠓⠟⠺⢷⡖⠚⢲⣼⣺⣿⠃⠠⠰⠃⠀⢀⠆⣔⢰⣢⢎⠐⢎⣡⠢⠘⡞⢿⢻⣶⣶⠢⣶⡜⡏⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠘⣴⠀⣾⣻⠻⣷⠀⠀⠀⠀⠀⣰⠀⣴⠘⢙⠉⡆⡆⡟⠀⠀⠃⠀⠸⡛⢻⣶⡆⠀⠄⠀⎥
⎢⡀⠄⠠⠀⣀⢀⣀⡔⣿⣻⡯⢷⣿⢿⣿⠤⢣⡰⠀⢍⠘⠃⠐⠈⣆⢀⢹⣷⢳⣄⢸⡿⣘⠿⠈⠉⣿⣿⣗⣿⎥
⎣⠂⠕⠨⠶⠙⠊⠳⠢⣿⢿⠍⢵⣽⢹⣿⠉⣗⢹⠀⢈⠸⠄⠤⠀⣾⠠⢰⣷⠝⢿⡶⠿⠏⠉⠀⠁⣽⣽⣵⣿⎦

In [11]:
f = FEM.loadVector(problem, [load])

2588-element Vector{Float64}:
 0.33333333333333326
 0.0
 0.0
 0.0
 0.0
 0.0
 0.33333333333333326
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
M = FEM.massMatrix(problem)

2588×2588 SparseMatrixCSC{Float64, Int64} with 2588 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⎦

In [13]:
C = 4e-6 * K

2588×2588 SparseMatrixCSC{Float64, Int64} with 77152 stored entries:
⎡⣻⣾⣳⣍⠀⠁⢈⢿⣸⢸⡓⢰⣸⠨⡥⠶⠎⠦⠒⠂⠀⠐⢶⡰⣜⣓⣚⠀⠈⠏⢼⡠⡁⠀⠀⠀⠀⠌⢌⠄⎤
⎢⡝⢾⠱⣦⡀⣀⠀⠹⣼⢸⢉⠸⣼⡐⡖⠚⠃⠓⠉⢁⡀⠈⠳⠜⠮⡩⢭⠀⠀⠃⡚⡔⡄⢀⠀⠀⠀⠂⢢⡆⎥
⎢⠄⠀⠀⢨⣻⢞⡳⡅⡷⡃⢬⡳⣿⠀⣟⢼⡣⣋⡁⠠⣝⠲⠀⣉⠀⡇⠠⢈⠀⠀⠀⣟⣽⠄⠀⠀⠀⢘⡳⠀⎥
⎢⣦⣔⣄⡀⠝⠮⠱⣦⢹⣥⣄⣹⣽⣠⣯⣾⣵⣁⣀⡄⠈⢙⣀⣀⣲⣧⣤⣦⠠⣶⣀⣏⢾⣆⣀⠀⢀⠼⠹⡂⎥
⎢⣒⣚⣒⣛⠽⠫⠗⣶⣿⣿⣊⡟⣿⣻⣿⣾⣷⣿⠁⡈⠹⠛⣀⠋⢛⡋⢙⣚⣿⣿⣿⣿⣸⠉⠐⠛⣿⣻⣿⣟⎥
⎢⢙⣈⣃⡐⢦⡳⣄⣹⣮⠼⣻⣾⢿⣿⣭⡉⠌⠠⣟⡛⠦⣴⣂⣶⣾⣿⡛⡿⠤⠤⢢⠲⣘⣶⣾⣻⢯⣏⢇⣅⎥
⎢⡒⡚⢒⠻⠛⠛⠓⣻⣿⣻⣿⣷⣿⣿⡛⣚⣒⣃⠀⡽⢃⡼⣺⢿⡿⣝⣿⣿⣘⣛⣟⣲⣾⣾⢿⣦⣿⣟⣗⣛⎥
⎢⢡⡏⣸⠉⣛⣝⣫⣿⣻⣿⡇⠻⣻⢨⣿⣿⣭⣹⠀⠐⢘⠁⣵⠀⠛⢚⠘⠛⢿⣻⠿⡟⠉⡀⠀⠀⠛⡟⡟⠛⎥
⎢⠪⡅⢭⠀⡭⢪⠕⢻⣽⣿⠂⡁⠼⢸⣇⣻⣿⢟⡀⠀⡨⠑⠈⠀⠀⠐⠀⢀⢇⢿⣪⢫⠴⠂⠀⠀⢉⡲⣝⣙⎥
⎢⠸⠀⠇⢀⠁⡈⠀⠼⡁⠠⣿⠹⣄⡤⢀⠀⠀⠈⣿⣿⡀⠰⠇⢨⠰⡠⣧⡯⠀⠀⡀⡠⠀⢀⢀⣠⡄⢄⡀⢀⎥
⎢⢀⠀⡀⠈⢳⡙⣆⢀⣷⠂⢈⣧⣉⡴⠖⠐⢆⠊⢀⡈⡻⣮⡀⢉⡉⣏⢉⣈⠀⠒⠁⠲⢈⢥⢀⣤⠶⠀⠒⠆⎥
⎢⢘⡳⣙⠆⡄⢠⠀⢸⡤⠘⢨⣼⣾⣞⠑⠛⠂⠀⡉⣁⡄⢈⣻⣾⣾⣤⡄⣨⡠⠣⡙⠙⠰⣲⣖⢀⡐⠀⠀⠃⎥
⎢⢶⢹⡎⡣⠤⠤⠼⣾⡿⠰⣾⣿⣟⢯⣻⢀⢀⠀⠐⡢⡧⢬⠚⣿⣻⣾⣷⣛⢉⣐⣉⠂⢊⠑⠣⠤⠈⢙⠚⡛⎥
⎢⠚⠘⠃⠓⡀⢂⠠⣿⣳⢰⣿⡬⣿⣿⣶⠀⠀⢀⡭⡿⡃⢰⡀⣩⣽⢻⢿⣷⡀⠀⢓⣀⠎⣱⣬⠭⢷⣶⢴⣶⎥
⎢⡦⠄⠤⠀⠀⠀⢠⣦⣿⣿⠀⡇⣶⢸⣿⣳⣭⣕⠀⠀⢠⠀⠤⡊⢃⢰⠀⠈⣿⣿⣼⡍⣈⠂⠀⠀⠙⢶⣷⣅⎥
⎢⠒⡳⢚⠬⣤⢤⡤⢼⣿⣿⢨⡒⢻⣹⣿⠧⡮⣚⠀⡨⢡⡀⣗⠈⠣⠘⠙⢰⡖⠿⣛⣼⣾⣍⠉⠀⣶⡶⣼⡏⎥
⎢⠁⠈⠀⢉⠓⠟⠺⢷⡖⠚⢲⣼⣺⣿⠃⠠⠰⠃⠀⢀⠆⣔⢰⣢⢎⠐⢎⣡⠢⠘⡞⢿⢻⣶⣶⠢⣶⡜⡏⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠘⣴⠀⣾⣻⠻⣷⠀⠀⠀⠀⠀⣰⠀⣴⠘⢙⠉⡆⡆⡟⠀⠀⠃⠀⠸⡛⢻⣶⡆⠀⠄⠀⎥
⎢⡀⠄⠠⠀⣀⢀⣀⡔⣿⣻⡯⢷⣿⢿⣿⠤⢣⡰⠀⢍⠘⠃⠐⠈⣆⢀⢹⣷⢳⣄⢸⡿⣘⠿⠈⠉⣿⣿⣗⣿⎥
⎣⠂⠕⠨⠶⠙⠊⠳⠢⣿⢿⠍⢵⣽⢹⣿⠉⣗⢹⠀⢈⠸⠄⠤⠀⣾⠠⢰⣷⠝⢿⡶⠿⠏⠉⠀⠁⣽⣽⣵⣿⎦

In [14]:
FEM.applyBoundaryConditions!(problem, K, M, C, f, [supp]);

In [15]:
Tₘᵢₙ = FEM.smallestPeriodTime(K, M)

2.745273270892753e-7

In [16]:
q = FEM.solveDisplacement(K, f)

2588-element Vector{Float64}:
  0.0005387852269263553
 -7.61154376898717e-6
  0.0
  0.0
  0.0
  0.0
  0.00053876250692672
  7.38845657550854e-6
  0.00021869445637258067
 -9.836833553490047e-8
  ⋮
 -7.853248222757615e-7
  3.142137965855278e-5
  3.66100795484183e-6
  2.9191277887710847e-5
 -4.048479977032783e-6
  0.0003092911248420364
 -3.535508653989133e-6
  0.00022889873554789465
 -3.545778249045908e-6

In [17]:
dim, dim = size(K)
u0 = zeros(dim)
v0 = zeros(dim)
FEM.initialDisplacement!(problem, "supp", u0, ux=0)
FEM.initialVelocity!(problem, "body", v0, vx=1000)
FEM.initialVelocity!(problem, "supp", v0, vx=0)
f = zeros(dim)

2588-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

$$\mathbf{M}\ddot{\mathbf{q}}+\mathbf{C}\dot{\mathbf{q}}+\mathbf{K}\mathbf{q}=\mathbf{f}$$
$$\mathbf{M}\ddot{\mathbf{q}}_n+\mathbf{C}\dot{\mathbf{q}}_n+\mathbf{K}\mathbf{q}_n=\mathbf{f}_n$$
$$\ddot{\mathbf{q}}_n = \frac{\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1}}{\Delta t^2}$$
$$\dot{\mathbf{q}}_n = \frac{\mathbf{q}_{n}-\mathbf{q}_{n-1}}{\Delta t}$$
$$\mathbf{M}\frac{\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1}}{\Delta t^2}+\mathbf{C}\frac{\mathbf{q}_{n}-\mathbf{q}_{n-1}}{\Delta t}+\mathbf{K}\mathbf{q}_n=\mathbf{f}_n$$
$$\mathbf{M}(\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1})+\Delta t\mathbf{C}(\mathbf{q}_{n}-\mathbf{q}_{n-1})+\Delta t^2\mathbf{K}\mathbf{q}_n=\Delta t^2\mathbf{f}_n$$
$$\mathbf{M}\mathbf{q}_{n+1}=\left(2\mathbf{M}-\Delta t\mathbf{C}-\Delta t^{2}\mathbf{K}\right)\mathbf{q}_{n}+(\Delta t\mathbf{C}-\mathbf{M})\mathbf{q}_{n-1}+\Delta t^{2}\mathbf{f}_{n}$$

In [18]:
c = √(problem.E / problem.ρ)
ξₘₐₓ = 1e-1
β = ξₘₐₓ * Tₘᵢₙ / π
C = β * K
u, v, t = FEM.CDM(K, M, C, f, u0, v0, base / c * 2, Tₘᵢₙ / π * (√(1 + ξₘₐₓ^2) - ξₘₐₓ) * 1.0)
size(u)

(2588, 502)

In [19]:
S = FEM.solveStress(problem, q)

Main.FEM.StressField([[-0.9663738559958376; 0.036019482518852695; … ; 0.0; 0.0;;], [-0.9663975402210883; -0.03620082482717857; … ; 0.0; 0.0;;], [-0.999999998423474; 7.777042648359681e-9; … ; 0.0; 0.0;;], [-0.87964454316323; 0.06896142305257648; … ; 0.0; 0.0;;], [-0.8801542074254544; -0.07019244514658929; … ; 0.0; 0.0;;], [-1.5954850603594855; 0.19012632658749282; … ; 0.0; 0.0;;], [-0.8579902966017583; 0.09774510912011238; … ; 0.0; 0.0;;], [-0.8585320776106208; -0.09817032661196436; … ; 0.0; 0.0;;], [-1.00000000822951; -3.485584050288554e-8; … ; 0.0; 0.0;;], [-1.0574252542179057; 0.005285812589493098; … ; 0.0; 0.0;;]  …  [-1.0000000132997389; 2.487217329797056e-8; … ; 0.0; 0.0;;], [-1.016704921365176; 0.0061670456460195386; … ; 0.0; 0.0;;], [-1.0000000049286855; -3.6813000564572818e-9; … ; 0.0; 0.0;;], [-1.000000007040101; 3.11685293793835e-9; … ; 0.0; 0.0;;], [-1.0189183055077116; -0.008923567398218382; … ; 0.0; 0.0;;], [-1.000000008884062; 2.2052880688772945e-9; … ; 0.0; 0.0;;], [-0.9

In [20]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [21]:
uvec = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
s = FEM.showStressResults(problem, S, "s", name="σ", visible=false, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=true)
sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)
vvec = FEM.showDoFResults(problem, v, t=t, "uvec", name="v(t)", visible=true)
gmsh.view.option.setNumber(vvec, "NormalRaise", 0.03)

"uvec..ok"

"ux..ok"

"uy..ok"

"uz..ok"

"s..ok"

...

"sxy..ok"

"syz..ok"

"szx..ok"

"uvec..ok"

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


In [22]:
#SS = []
#for i in 1:10
#    S = FEM.solveStress(problem, u[:, i])
#    push!(SS, S)
#end
#SS

In [23]:
#pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
#pp2 = gmsh.model.occ.addPoint(base, height / 2, 0)
#pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

#pl1 = gmsh.model.occ.addLine(pp1, pp2)
#pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


#gmsh.model.occ.synchronize()

In [24]:
sts = ceil(Int64, (base / c * 2) / 6 / (Tₘᵢₙ / π * (√(1 + ξₘₐₓ^2) - ξₘₐₓ)))
display(sts)
Sp = FEM.solveStress(problem, u[:, sts])
sp = FEM.showStressResults(problem, Sp, "s", name="σ at t", visible=false, smooth=false);

84

"s..ok"

In [25]:
Sanim = FEM.solveStress(problem, u[:, 1:sts])
sanim = FEM.showStressResults(problem, Sanim, "s", t=t[1:sts], name="σ anim", visible=false, smooth=false);

"s..ok"

In [26]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [27]:
ENV["LIBGL_ALWAYS_SOFTWARE"] = "false"
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240121
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [28]:
gmsh.finalize()